In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv").drop(columns=['Soil_Type7', 'Soil_Type15']) 
test_df = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv").drop(columns=['Soil_Type7', 'Soil_Type15']) 

In [ ]:
df.head()

## check for null values

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby('Cover_Type').size()

In [ ]:
X = df.drop(['Id','Cover_Type'],axis = 1)
y = df.Cover_Type
len(y.unique())

In [ ]:
X.head(2)

In [ ]:
sns.histplot(X.Elevation)

In [ ]:
sns.histplot(X.Aspect)

In [ ]:
sns.histplot(X.Slope)

In [ ]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
sc = StandardScaler()
le = LabelEncoder()
y = le.fit_transform(y)
X = sc.fit_transform(X)

In [ ]:
set(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size = 0.02,random_state = 42)
X_test,X_,y_test,y_ = train_test_split(X_val,y_val,test_size = 0.000002,random_state = 42)

In [ ]:
len(X_train),len(X_val),len(X_test)

In [ ]:
X_train.shape,y_train.shape

In [ ]:
EPOCHS = 90 
VERBOSE = 2 
RUNS = 1 
BATCH_SIZE = 512 

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    pass
print('Num',strategy.num_replicas_in_sync)

In [ ]:
def model_fun(X):
    il = tf.keras.layers.Input(shape=(X.shape[-1]))
    x = tf.keras.layers.Dense(128, activation='relu')(il)
    x1 = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x1)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(tf.keras.layers.Concatenate()([x, x1]))
    x = tf.keras.layers.Dense(units=64, activation='relu')(x) 
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    output = tf.keras.layers.Dense(7, activation="softmax")(x)
    model = tf.keras.Model(inputs = il,outputs = output)
    return model
    

In [ ]:
lr =tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, verbose=VERBOSE)
es = tf.keras.callbacks.EarlyStopping(monitor="val_acc", patience=10, verbose=VERBOSE, mode="max", restore_best_weights=True)

In [ ]:
with strategy.scope():
    model = model_fun(X_train)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])
    history = model.fit(X_train,y_train,
                        verbose=VERBOSE,
                        shuffle=True,
                        batch_size=BATCH_SIZE,epochs=EPOCHS
                        ,use_multiprocessing=True,
                        callbacks=[lr,es],
                        validation_data=(X_val,y_val))

In [ ]:
y_predict = model.predict(X_test)
y_predict = le.inverse_transform(np.argmax(y_predict,axis = 1))

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score
plt.figure(figsize=(10,10))
cm = confusion_matrix(y_test,y_predict)
sns.heatmap(cm,annot=True,fmt='.2f')

In [ ]:
accuracy_score(y_test,y_predict)*100

In [ ]:
test_df.head()

In [ ]:
Ids = test_df.Id
test_df.drop('Id',axis = 1,inplace = True)

In [ ]:
sub_x = sc.fit_transform(test_df)

In [ ]:
sub_x.shape

In [ ]:
sub_predict = model.predict(sub_x)
sub_predict = le.inverse_transform(np.argmax(sub_predict,axis = 1))

In [ ]:
sub = pd.DataFrame()

In [ ]:
sub['Id'] = Ids
sub['Cover_Type'] = sub_predict

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission_dec.csv',index = False)